<h1>IBM Applied Data Science Capstone Course by Coursera</h1>

<h3>Opening a New Shopping Mall in India, Bangalore</h3>

<ul>
<li>Build a dataframe of neighborhoods in India, Bangalore by web scraping the data from Wikipedia page</li>
<li>Get the geographical coordinates of the neighborhoods</li>
<li>Obtain the venue data for the neighborhoods from Foursquare API</li>
<li>Explore and cluster the neighborhoods</li>
<li>Select the best cluster to open a new shopping mall</li>
</ul>

In [110]:
import numpy as np 
import pandas as pd 
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
import json
from geopy.geocoders import Nominatim 
import geocoder 
import requests 
from bs4 import BeautifulSoup 
from pandas.io.json import json_normalize 
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium 

print("Libraries imported.")

Libraries imported.


In [83]:
d1 = pd.read_html("https://en.wikipedia.org/wiki/List_of_neighbourhoods_in_Bangalore")[0]
d2 = pd.read_html("https://en.wikipedia.org/wiki/List_of_neighbourhoods_in_Bangalore")[1]
d3 = pd.read_html("https://en.wikipedia.org/wiki/List_of_neighbourhoods_in_Bangalore")[2]
d4 = pd.read_html("https://en.wikipedia.org/wiki/List_of_neighbourhoods_in_Bangalore")[3]
d5 = pd.read_html("https://en.wikipedia.org/wiki/List_of_neighbourhoods_in_Bangalore")[4]
d6 = pd.read_html("https://en.wikipedia.org/wiki/List_of_neighbourhoods_in_Bangalore")[5]
d7 = pd.read_html("https://en.wikipedia.org/wiki/List_of_neighbourhoods_in_Bangalore")[6]
d8 = pd.read_html("https://en.wikipedia.org/wiki/List_of_neighbourhoods_in_Bangalore")[7]

In [85]:
data = pd.concat([d1.Name,d2.Name,d3.Name,d4.Name,d5.Name,d6.Name,d7.Name,d8.Name], ignore_index=True)
bl_df = pd.DataFrame({"Neighborhood": data})
bl_df.head()

,Neighborhood
0,Cantonment area
1,Domlur
2,Indiranagar
3,Rajajinagar
4,Malleswaram


In [90]:
bl_df.shape

(65, 1)

<h3>3. Get the geographical coordinates</h3>

In [95]:
def get_coords(n):
    return geocoder.arcgis(f'{n}, Karnataka, Bangalore').latlng

In [98]:
loc = bl_df['Neighborhood'].tolist()
coords = [get_coords(code) for code in loc]
print("Coordinates are obtained.")

Coordinates are obtained.


In [102]:
# a new dataframe to store coordinates
coords_arcgis = bl_df.copy()

# add columns for latitudes and longitudes
coords_arcgis['Latitude'] = [coord[0] for coord in coords]
coords_arcgis['Longitude'] = [coord[1] for coord in coords]
bl_df = coords_arcgis.copy()

In [105]:
bl_df.head()

,Neighborhood,Latitude,Longitude
0,Cantonment area,12.96618,77.586900
1,Domlur,12.94329,77.656020
2,Indiranagar,13.03006,77.495260
3,Rajajinagar,13.00544,77.556930
4,Malleswaram,13.00632,77.568405


In [107]:
# save the DataFrame as CSV file
bl_df.to_csv("bl_df.csv", index=False)

<h3>4. Create a map of Karnataka with neighborhoods superimposed on top</h3>

In [112]:
address = 'Karnataka, Bangalore'

geolocator = Nominatim(user_agent="http")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Karnataka, Bangalore {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Karnataka, Bangalore 12.9791198, 77.5912997.


In [114]:
map_kl = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(bl_df['Latitude'], bl_df['Longitude'], bl_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_kl)  
    
map_kl

In [115]:
# save the map as HTML file
map_kl.save('map_kl.html')

<h3>5. Use the Foursquare API to explore the neighborhoods</h3>

In [117]:
CLIENT_ID = 'RJHSJA2APNIOA3JOV1NWT4X1ED4ZZHJ5PGBU0R2FM0YTF5UY' # your Foursquare ID
CLIENT_SECRET = 'RLXEIOVLJLG2FSUYVONWSBS0FCOKGTJDPBISKZENQA0NOO4Q' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: RJHSJA2APNIOA3JOV1NWT4X1ED4ZZHJ5PGBU0R2FM0YTF5UY
CLIENT_SECRET:RLXEIOVLJLG2FSUYVONWSBS0FCOKGTJDPBISKZENQA0NOO4Q


<h5>Get the top 100 venues that are within a radius of 2000 meters.</h5>

In [ ]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(bl_df['Latitude'], bl_df['Longitude'], bl_df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [119]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(3389, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Cantonment area,12.96618,77.5869,Lakshmi Nataraj Refreshments,12.967476,77.579468,South Indian Restaurant
1,Cantonment area,12.96618,77.5869,Kebabs & Kurries,12.966995,77.595613,Indian Restaurant
2,Cantonment area,12.96618,77.5869,JW Marriott Hotel Bengaluru,12.972362,77.595051,Hotel
3,Cantonment area,12.96618,77.5869,Toscano,12.971980,77.596066,Italian Restaurant
4,Cantonment area,12.96618,77.5869,Bliss Luxe Chocolate Lounge,12.971525,77.596201,Cupcake Shop


<h5>Check how many venues were returned for each neighorhood</h5>

In [120]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Anjanapura,6,6,6,6,6,6
Arekere,67,67,67,67,67,67
BTM Layout,80,80,80,80,80,80
Banashankari,100,100,100,100,100,100
Banaswadi,57,57,57,57,57,57
Basavanagudi,100,100,100,100,100,100
Basaveshwaranagar,50,50,50,50,50,50
Begur,15,15,15,15,15,15
Bellandur,75,75,75,75,75,75


<h5>Find out how many unique categories can be curated from all the returned venues</h5>

In [121]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 206 uniques categories.


In [122]:
# print out the list of categories
venues_df['VenueCategory'].unique()[:50]

array(['South Indian Restaurant', 'Indian Restaurant', 'Hotel',
       'Italian Restaurant', 'Cupcake Shop', 'Lounge',
       'Seafood Restaurant', 'Shopping Mall', 'Cocktail Bar',
       'French Restaurant', 'Sushi Restaurant', 'Japanese Restaurant',
       'Gym / Fitness Center', 'Park', 'Deli / Bodega',
       'Asian Restaurant', 'Ice Cream Shop', 'Café', 'Bagel Shop',
       'Brewery', 'Burger Joint', 'Karnataka Restaurant', 'Theater',
       'Breakfast Spot', 'American Restaurant', 'Cricket Ground',
       'Arcade', 'Bakery', 'Dessert Shop', 'Chinese Restaurant',
       'Restaurant', 'Pub', 'Bed & Breakfast', 'Sandwich Place',
       'Wine Bar', 'Gym', 'Spa', 'Coffee Shop', 'Hockey Arena',
       'Botanical Garden', 'Pizza Place', 'Mexican Restaurant', 'Stadium',
       'Steakhouse', 'Snack Place', 'Rajasthani Restaurant',
       'Mediterranean Restaurant', 'Movie Theater', 'Track Stadium',
       'Bookstore'], dtype=object)

In [123]:
# check if the results contain "Shopping Mall"
"Neighborhood" in venues_df['VenueCategory'].unique()

True

<h3>6. Analyze Each Neighborhood</h3>

In [126]:
# one hot encoding
bl_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
bl_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [bl_onehot.columns[-1]] + list(bl_onehot.columns[:-1])
bl_onehot = bl_onehot[fixed_columns]

print(bl_onehot.shape)
bl_onehot.head()

(3389, 207)


,Neighborhoods,Accessories Store,Afghan Restaurant,American Restaurant,Andhra Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Badminton Court,Bagel Shop,Bakery,Bar,Basketball Court,Bed & Breakfast,Beer Bar,Beer Garden,Bengali Restaurant,Bike Shop,Bistro,Boarding House,Bookstore,Botanical Garden,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Campground,Candy Store,Cantonese Restaurant,Capitol Building,Caribbean Restaurant,Chaat Place,Chettinad Restaurant,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cricket Ground,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Financial or Legal Service,Fish & Chips Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden Center,Gas Station,Gastropub,General Entertainment,German Restaurant,Golf Course,Grocery Store,Gym,Gym / Fitness Center,Halal Restaurant,Historic Site,Hockey Arena,Hookah Bar,Hotel,Hotel Bar,Hotel Pool,Hyderabadi Restaurant,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karnataka Restaurant,Kerala Restaurant,Korean Restaurant,Lake,Light Rail Station,Lighthouse,Liquor Store,Lounge,Maharashtrian Restaurant,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Monument / Landmark,Motorcycle Shop,Movie Theater,Moving Target,Multicuisine Indian Restaurant,Multiplex,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,North Indian Restaurant,Office,Optical Shop,Other Great Outdoors,Outlet Store,Pakistani Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Pharmacy,Photography Lab,Pizza Place,Platform,Playground,Plaza,Pool,Pub,Punjabi Restaurant,Racetrack,Rajasthani Restaurant,Recreation Center,Resort,Restaurant,Road,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,South Indian Restaurant,Spa,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Szechuan Restaurant,Tea Room,Tennis Stadium,Tex-Mex Restaurant,Thai Restaurant,Theater,Tibetan Restaurant,Toll Booth,Tourist Information Center,Toy / Game Store,Track Stadium,Trail,Train Station,Travel & Transport,Turkish Restaurant,Udupi Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Women's Store
0,Cantonment area,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Cantonment area,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Cantonment area,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

<h5>Lets group rows by neighborhood and by taking the mean of the frequency of occurrence of each category</h5>

In [129]:
bl_grouped = bl_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(bl_grouped.shape)
bl_grouped

(64, 207)


,Neighborhoods,Accessories Store,Afghan Restaurant,American Restaurant,Andhra Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Badminton Court,Bagel Shop,Bakery,Bar,Basketball Court,Bed & Breakfast,Beer Bar,Beer Garden,Bengali Restaurant,Bike Shop,Bistro,Boarding House,Bookstore,Botanical Garden,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Campground,Candy Store,Cantonese Restaurant,Capitol Building,Caribbean Restaurant,Chaat Place,Chettinad Restaurant,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cricket Ground,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Financial or Legal Service,Fish & Chips Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden Center,Gas Station,Gastropub,General Entertainment,German Restaurant,Golf Course,Grocery Store,Gym,Gym / Fitness Center,Halal Restaurant,Historic Site,Hockey Arena,Hookah Bar,Hotel,Hotel Bar,Hotel Pool,Hyderabadi Restaurant,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karnataka Restaurant,Kerala Restaurant,Korean Restaurant,Lake,Light Rail Station,Lighthouse,Liquor Store,Lounge,Maharashtrian Restaurant,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Monument / Landmark,Motorcycle Shop,Movie Theater,Moving Target,Multicuisine Indian Restaurant,Multiplex,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,North Indian Restaurant,Office,Optical Shop,Other Great Outdoors,Outlet Store,Pakistani Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Pharmacy,Photography Lab,Pizza Place,Platform,Playground,Plaza,Pool,Pub,Punjabi Restaurant,Racetrack,Rajasthani Restaurant,Recreation Center,Resort,Restaurant,Road,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,South Indian Restaurant,Spa,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Szechuan Restaurant,Tea Room,Tennis Stadium,Tex-Mex Restaurant,Thai Restaurant,Theater,Tibetan Restaurant,Toll Booth,Tourist Information Center,Toy / Game Store,Track Stadium,Trail,Train Station,Travel & Transport,Turkish Restaurant,Udupi Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Women's Store
0,Anjanapura,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.166667,0.00,0.000000,0.000000,0.166667,0.00,0.00,0.00,0.00,0.00000,0.000000,0.000000,0.00,0.000000,0.00000,0.166667,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.166667,0.000000,0.000000,0.000000,0.000000,0.0000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.00000,0.00

In [131]:
len(bl_grouped[bl_grouped["Shopping Mall"] > 0])

19

<h5>Create a new DataFrame for Shopping Mall data only</h5>

In [132]:
bl_mall = bl_grouped[["Neighborhoods","Shopping Mall"]]

In [134]:
bl_mall.head()

,Neighborhoods,Shopping Mall
0,Anjanapura,0.000000
1,Arekere,0.014925
2,BTM Layout,0.000000
3,Banashankari,0.020000
4,Banaswadi,0.000000


<h3>7. Cluster Neighborhoods</h3>

Run k-means to cluster the neighborhoods in Bangalore into 3 clusters.

In [136]:
# set number of clusters
kclusters = 3

bl_clustering = bl_mall.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(bl_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 2, 0, 2, 0, 0, 0, 0, 2, 0])

In [138]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
bl_merged = bl_mall.copy()

# add clustering labels
bl_merged["Cluster Labels"] = kmeans.labels_

In [139]:
bl_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
bl_merged.head()

,Neighborhood,Shopping Mall,Cluster Labels
0,Anjanapura,0.000000,0
1,Arekere,0.014925,2
2,BTM Layout,0.000000,0
3,Banashankari,0.020000,2
4,Banaswadi,0.000000,0


In [140]:
# merge bangalore_grouped with bangalore_data to add latitude/longitude for each neighborhood
bl_merged = bl_merged.join(bl_df.set_index("Neighborhood"), on="Neighborhood")

print(bl_merged.shape)
bl_merged.head() # check the last columns!


(65, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Anjanapura,0.000000,0,12.858110,77.559090
1,Arekere,0.014925,2,12.885680,77.596680
2,BTM Layout,0.000000,0,12.914950,77.610010
3,Banashankari,0.020000,2,12.922310,77.569880
4,Banaswadi,0.000000,0,13.019527,77.655028


In [141]:
# sort the results by Cluster Labels
print(bl_merged.shape)
bl_merged.sort_values(["Cluster Labels"], inplace=True)
bl_merged

(65, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Anjanapura,0.000000,0,12.858110,77.559090
31,Koramangala,0.000000,0,12.920040,77.625460
62,Yelahanka,0.000000,0,13.099310,77.592590
34,Kumaraswamy Layout,0.000000,0,12.898190,77.559270
36,Madiwala,0.000000,0,12.920520,77.620900
37,Mahadevapura,0.000000,0,12.994090,77.666330
39,Malleswaram,0.010526,0,13.006320,77.568405
40,Marathahalli,0.000000,0,12.954660,77.707520
41,Mathikere,0.000000,0,13.032350,77.558660
42,Nagarbhavi,0.000000,0,12.956240,77.509360


<h5>Lets visualize the resulting clusters</h5>

In [142]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(bl_merged['Latitude'], bl_merged['Longitude'], bl_merged['Neighborhood'], bl_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [143]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

<h5>8. Examine Clusters</h5>

<h6>Cluster 0</h6>

In [144]:
bl_merged.loc[bl_merged['Cluster Labels'] == 0]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Anjanapura,0.000000,0,12.858110,77.559090
31,Koramangala,0.000000,0,12.920040,77.625460
62,Yelahanka,0.000000,0,13.099310,77.592590
34,Kumaraswamy Layout,0.000000,0,12.898190,77.559270
36,Madiwala,0.000000,0,12.920520,77.620900
37,Mahadevapura,0.000000,0,12.994090,77.666330
39,Malleswaram,0.010526,0,13.006320,77.568405
40,Marathahalli,0.000000,0,12.954660,77.707520
41,Mathikere,0.000000,0,13.032350,77.558660
42,Nagarbhavi,0.000000,0,12.956240,77.509360


<h6>Cluster 1</h6>

In [147]:
bl_merged.loc[bl_merged['Cluster Labels'] == 1]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
25,Jalahalli,0.166667,1,13.0545,77.52658


<h6>Cluster 2</h6>

In [149]:
bl_merged.loc[bl_merged['Cluster Labels'] == 2]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
55,Ulsoor,0.020000,2,12.98908,77.62795
3,Banashankari,0.020000,2,12.92231,77.56988
1,Arekere,0.014925,2,12.88568,77.59668
29,Kammanahalli,0.027397,2,13.01430,77.63685
45,Padmanabhanagar,0.015385,2,12.91814,77.55940
43,Nandini Layout,0.025641,2,13.01481,77.53891
38,Mahalakshmi Layout,0.016949,2,13.01635,77.54481
22,Hulimavu,0.018182,2,12.88064,77.60147
35,Lingarajapuram,0.030769,2,13.00548,77.62597
33,Krishnarajapuram,0.036364,2,13.00039,77.68368


<h3>Observations:</h3>
<h6>Most of the shopping malls are concentrated in the central area of Bangalore, with the highest number in cluster 2 and moderate number in cluster 0. On the other hand, cluster 1 has very low number to totally no shopping mall in the neighborhoods. This represents a great opportunity and high potential areas to open new shopping malls as there is very little to no competition from existing malls. Meanwhile, shopping malls in cluster 2 are likely suffering from intense competition due to oversupply and high concentration of shopping malls. From another perspective, this also shows that the oversupply of shopping malls mostly happened in the central area of the city, with the suburb area still have very few shopping malls. Therefore, this project recommends property developers to capitalize on these findings to open new shopping malls in neighborhoods in cluster 1 with little to no competition. Property developers with unique selling propositions to stand out from the competition can also open new shopping malls in neighborhoods in cluster 0 with moderate competition. Lastly, property developers are advised to avoid neighborhoods in cluster 2 which already have high concentration of shopping malls and suffering from intense competition. </h6>